# Assignment 9

In [1]:
import numpy as np
import pandas as pd

### Part A

Consider a two period binomial model. Suppose after each period, the stock will either rise in value by ```r``` percent, or fall in value by ```f``` percent. Currently, the stock price is ```P```, and the risk free rate is ```rf```.

Write the function ```european_put```, which calculates the value of a one year European Put option with a strike price of ```K```. 

In [2]:
def european_put(P, K, r, f, rf):
    rf6Month = np.sqrt(1 + rf) - 1
    s1Plus = P * (1 + r)
    s1Minus = P * (1 + f)
    
    s2Top = s1Plus * (1 + r)
    s2Second = s1Plus * (1 + f)
    s2Third = s1Minus * (1 + r)
    s2Bottom = s1Minus * (1 + f)
    
    pStar = ((P * (1 + rf6Month)) - s1Minus) / (s1Plus - s1Minus)
    p2Top = max(K - s2Top, 0)
    p2Second = max(K - s2Second, 0)
    p2Third = max(K - s2Third, 0)
    p2Bottom = max(K - s2Bottom, 0)
    
    p1Plus = ((pStar * p2Top) + ((1 - pStar) * p2Second)) / (1 + rf6Month)
    p1Minus = ((pStar * p2Third) + ((1 - pStar) * p2Bottom)) / (1 + rf6Month)
    
    p0 = ((pStar * p1Plus) + ((1 - pStar) * p1Minus)) / (1 + rf6Month)
    return p0
            
# european_put(40,40, 0.25, -0.2, 0.1) should return 2.616...

Write the function ```american_put```, which calculates the value of a one year American Put option with a strike price of ```K```. 

In [3]:
def american_put(P, K, r, f, rf):
    rf6Month = np.sqrt(1 + rf) - 1
    s1Plus = P * (1 + r)
    s1Minus = P * (1 + f)
    
    s2Top = s1Plus * (1 + r)
    s2Second = s1Plus * (1 + f)
    s2Third = s1Minus * (1 + r)
    s2Bottom = s1Minus * (1 + f)
    
    pStar = ((P * (1 + rf6Month)) - s1Minus) / (s1Plus - s1Minus)
    p2Top = max(K - s2Top, 0)
    p2Second = max(K - s2Second, 0)
    p2Third = max(K - s2Third, 0)
    p2Bottom = max(K - s2Bottom, 0)
    
    p1Plus = max(K - s1Plus, ((pStar * p2Top) + ((1 - pStar) * p2Second)) / (1 + rf6Month))
    p1Minus = max(K - s1Minus, ((pStar * p2Third) + ((1 - pStar) * p2Bottom)) / (1 + rf6Month))
    
    p0 = max(K - P, ((pStar * p1Plus) + ((1 - pStar) * p1Minus)) / (1 + rf6Month))
    return p0

# american_put(40,40, 0.25, -0.2, 0.1) should return 3.410...

### Part B

In addition to simple calls and puts, a wide variety of exotic options are traded in over-the-counter markets. In most cases, there are no analytic formulas available to value these options, and so numerical techniques must be used. One example of these techniques are Monte Carlo methods. These are particularly useful in certain contexts, such as when option payoffs depend not only on the stock price at expiration but also upon the history of the stock price sampled at various points during the life of the option. This question will explore some basic examples, and it will require relatively large number of paths. Assume that the stock price will be needed at the end of each of the next 12 months, in addition to S0 today, and start by simulating `N` such stock price paths. In particular:

•	You have been provided the function ```gen_random_vars```, which generates random variables that follow a standard normal distribution (Note that you should never actually call ```gen_random_vars``` in any of your functions, this is simply meant to help give you inputs to help test your code). This outputs a table with `N` rows and 12 columns. Denote the value of the random variable on path i for month j by zi,j where i = 1, . . . ,`N` and j = 1, . . . ,12.

Write the function ```create_paths```, which takes as an argument a DataFrame ```random_vars```, the output of the function ```gen_random_vars```, and ```S0```, the initial stock price.

This function will generate a DataFrame with `N` stock price paths $S_{i,j}$ where $i = 1, . . . ,N$ and $j = 0, . . . ,12$. In each case, $S_{i,0} =$ ```S_0```. For the remaining entries in this table, define $∆t = 1/12$ and set

$$ S_{i,j} = S_{i, j-1} \exp [(r - \sigma^2 / 2) \Delta t + \sigma \sqrt{\Delta t} z_{i,j}]$$

 
for i = 1, . . . ,`N` and j = 1, . . . ,12. (If you are wondering why we are using the risk-free rate r here instead of the expected rate of return on the underlying stock, it is because we are using risk-neutral valuation.) 

Note that throughout the rest of this question, $S_T$ on each simulated path i is defined to be equal to $S_{i,12}$.



In [4]:
def gen_random_vars(SEED, N):
    rng = np.random.default_rng(SEED)

    zArr = np.array([rng.normal(size=N)])
    for i in range(11):
        z = rng.normal(size=N)
        zArr = np.r_[zArr,[z]]
    

    randomVariables = pd.DataFrame(zArr.transpose())
    return randomVariables

In [5]:
def create_paths(random_vars, S0, rf, sigma):
    # Do not rename the columns or rows of the outputted dataframe. 
    # The outputted DataFrame should have N rows and 13 columns.
    N = len(random_vars.index)
    df = pd.DataFrame(0.0, index=pd.RangeIndex(start=0, stop=N, step=1), columns=pd.RangeIndex(start=0, stop=13, step=1))
    
    df[0] = S0
    dt = 1 / 12
    for i in range(N):
        for j in range(1, 13):
            firstHalf = (rf - ((sigma ** 2) / 2)) * dt
            df.iloc[i, j] = df.iloc[i, j - 1] * np.exp(firstHalf + (sigma * np.sqrt(dt) * random_vars.iloc[i, j - 1]))
    
    return df

# If we let sample_paths = create_paths(r_vars, 72.25, 0.04 , 0.35),
# The last column of the first row of sample_paths should equal 71.56...

•	Write the function ```payoff_table```, which returns another DataFrame with `N` rows and 6 columns. The columns will be used to store discounted payoffs at expiration for various different options, as indicated below:

–	European call option, with payoff $max(S_T − K,0)$.

–	European put option, with payoff $max(K - S_T ,0)$.

–	Average price put option, with payoff $max(K − S^-,0)$, where
$S¯ = S_0 + S_1 + · · · + S_{12}$ .

–	Floating lookback call option, with payoff $S_T - S_{min}$, where $S_{min}$ on each path is the minimum of the stock prices $S_0, S_1, . . . , S_{12}$ on that path.

–	Up-and-out call option with barrier ```B```, with payoff $max(S_T - K,0)$ as long as none of the observed monthly stock prices on the given path are greater than or equal to ```B```. If any of the monthly stock prices are greater than or equal to B, then the payoff is zero.

–	Up-and-in call option with barrier ```B``` , with payoff $max(S_T - K,0)$ as long as at least one of the observed monthly stock prices on the given path is greater than or equal to ```B```. If this condition is not satisfied, the option payoff is zero.

Note that “discounted payoffs” implies that the entries in this table should have the payoff functions multiplied by $e^{−rT}$ , where T = 1.

This function will also take in a dataframe ```paths```, which is the output from the previous function ```create_paths```.

In [6]:
def payoff_table(paths, K, T, rf, B):
    euroCall = []
    euroPut = []
    avgPricePut = []
    floatLookback = []
    upAndOut = []
    upAndIn = []
    
    numCols = len(paths.columns)
    for i in range(len(paths.index)):
        rowSum = 0
        rowMin = float("inf")
        upAndOutValid = True
        upAndInValid = False
        for j in range(numCols):
            currPrice = paths.iloc[i, j]
            rowSum += currPrice
            
            if currPrice < rowMin:
                rowMin = currPrice
            
            if currPrice >= B:
                upAndOutValid = False
                upAndInValid = True
            
            if j == numCols - 1:
                avgPrice = rowSum / numCols
                
                euroCall.append(max(currPrice - K, 0) * np.exp(-rf * T))
                euroPut.append(max(K - currPrice, 0) * np.exp(-rf * T))
                avgPricePut.append(max(K - avgPrice, 0) * np.exp(-rf * T))
                floatLookback.append((currPrice - rowMin) * np.exp(-rf * T))
                
                if upAndOutValid:
                    upAndOut.append(max(currPrice - K, 0) * np.exp(-rf * T))
                else:
                    upAndOut.append(0)
                
                if upAndInValid:
                    upAndIn.append(max(currPrice - K, 0) * np.exp(-rf * T))
                else:
                    upAndIn.append(0)
    
    answer = pd.DataFrame(data = {
        "European Call" : euroCall,
        "European Put" : euroPut,
        "Average Price Put" : avgPricePut,
        "Floating Lookback" : floatLookback,
        "Up-and-out" : upAndOut,
        "Up-and-in" : upAndIn
    })
    
    return answer
            
# If we let sample_payoffs = payoff_table(sample_paths, 70, 1, 0.04, 90)
# The payoff of the first European Call should be 1.500...

Write the function ```statistics```, which takes as input a DataFrame ```payoffs```, formatted as one produced by ```payoff_table```, and displays the following statistics for each of the options above (below N is the number of simulated paths):

•	Mean discounted payoff µ. This is the estimated option premium.

•	Standard error of option premium ϕ. This is the standard deviation of the option payoff divided by √N .

•	Lower bound for 95% confidence interval for option premium, which is µ - 1.96ϕ.

•	Upper bound for 95% confidence interval for option premium, which is µ + 1.96ϕ.

The size of the returned DataFrame should have 6 rows and 4 columns. 

In [7]:
def statistics(payoffs):
    # Note that the index rows have been renamed here. Ensure that your rows match the order 
    # that the indices have been renamed as.
    numRows = len(payoffs.index)
    meanLst = []
    stdErrLst = []
    lowerBoundLst = []
    upperBoundLst = []
    
    for seriesName, series in payoffs.items():
        currMean = series.mean()
        currStdErr = series.std() / np.sqrt(numRows)
        
        meanLst.append(currMean)
        stdErrLst.append(currStdErr)
        lowerBoundLst.append(currMean - (1.96 * currStdErr))
        upperBoundLst.append(currMean + (1.96 * currStdErr))
    
    output = pd.DataFrame(data = {
        "Mean" : meanLst,
        "Standard Error" : stdErrLst,
        "Lower Bound" : lowerBoundLst,
        "Upper Bound" : upperBoundLst
    }).rename(index={0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}, errors="raise")
    
    return output

# If we call statistics(sample_payoffs), the mean of the European Puts should be 7.370...

One disadvantage of Monte Carlo methods is that they can be slow to converge. On the other hand, there are situations where they are the only methods that can be used. As a result, there has been a lot of research into variance reduction methods, in an effort to improve the rate of convergence. 

One of the simplest such approaches is called antithetic variates. This relies on the fact that if Z is a standard normal random variable, then so is -Z. To see how this can be used, generate another `N` stock price paths, but this time using $-z_{i,j}$ instead of $z_{i,j}$ throughout.

In order to accomplish this, write the function ```create_negative_paths```. This function has the same arguments as ```create_paths```, but creates the stock prices in the manner described above.

In [8]:
def create_negative_paths(random_vars, S0, rf, sigma):
    # Do not rename the columns or rows of the outputted dataframe. 
    # The outputted DataFrame should have N rows and 13 columns.
    N = len(random_vars.index)
    df = pd.DataFrame(0.0, index=pd.RangeIndex(start=0, stop=N, step=1), columns=pd.RangeIndex(start=0, stop=13, step=1))
    
    df[0] = S0
    dt = 1 / 12
    for i in range(N):
        for j in range(1, 13):
            firstHalf = (rf - ((sigma ** 2) / 2)) * dt
            negZ = -1 * random_vars.iloc[i, j - 1]
            df.iloc[i, j] = df.iloc[i, j - 1] * np.exp(firstHalf + (sigma * np.sqrt(dt) * negZ))
    
    return df

Write the function ```antithetic_statistics```.

In this function, you will use ```payoff_table``` to create two tables, one using paths generated from ```create_paths```, and one generated from ```create_negative_paths```.

Next, create a third table which contains the average payoff between the two tables.

Finally, call ```statistics``` on this third table. Return the table produced by ```statistics```.

If you compare this table with the output of `statistics` (as above, without creating two tables), you should notice that Mean, Lower Bound, and Upper Bound columns should have relatively similar values, while the Standard Error column has decreased significantly.

In [9]:
def antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    normalPaths = create_paths(random_vars, S0, rf, sigma)
    normalPayoffs = payoff_table(normalPaths, K, T, rf, B)
    
    negativePaths = create_negative_paths(random_vars, S0, rf, sigma)
    negativePayoffs = payoff_table(negativePaths, K, T, rf, B)
    
    avgPayoffs = normalPayoffs.copy()
    for i in range(len(avgPayoffs.index)):
        for j in range(len(avgPayoffs.columns)):
            avgPayoffs.iloc[i, j] = (avgPayoffs.iloc[i, j] + negativePayoffs.iloc[i, j]) / 2
    
    return statistics(avgPayoffs)